# Topicos de estudo para a AI

## Fases de um projeto de machine learning versus etapas do CRISP-DM

<img src="crisp.png" width=80%/>

## Análise Exploratória

- O que fazer **antes** da separação treino-teste e o que fazer **depois**

    - Antes: analise da natureza global das variaveis, independentemente uma das outras

        - Só coisa "ingênua", que não leva diretamente a construção de modelos

        - Análise de anomalias: valores faltantes, outliers, erros grosseiros

        - Variáveis contínuas:
        
            - Medidas descritivas (média, mediana, desvio padrão, etc)

            - Histogramas

        - Variáveis categóricas:

            - Frequências (`value_counts`)

    - Separação treino-teste:

        - Nada especial, só lembre de fixar o `random_state`

            > ```Python
            > 
            > from sklearn.model_selection import train_test_split
            > 
            > SEED = 42
            > 
            > X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
            > 
            > ```

    - Depois: tá liberado, mas só no conjunto de treino

        - Analisar dependencias entre variaveis, e entre elas e o target

        - Contínua versus contínua:

            - Medidas de correlação (Pearson, Spearman, Kendall)

            - Gráficos de espalhamento (*scatter plots*)

        - Categórica versus categórica

            - Teste de independencia: teste qui-quadrado de Pearson (*chi-square*). **Não foi visto em aula, não se preocupe.**

            - Tabela de contingência (`cross_tab`)

        - Categórica versus contínua

            - Boxplot da contínua para cada categoria

            - Teste de Kolmogorov-Smirnov entre valores da variável contínua para pares de categorias. **Não foi visto em aula, não se preocupe.**


In [ ]:
from sklearn.datasets import load_wine

data = load_wine(as_frame=True)
print(data.DESCR)

In [ ]:
dataset = data.data
dataset['target'] = data.target.astype('category')

dataset

In [ ]:
dataset.info()

In [ ]:
dataset \
    .select_dtypes(include='float64') \
    .describe() \
    .round(2) \
    .transpose()

In [ ]:
dataset \
    .select_dtypes(include='category') \
    .describe()

In [ ]:
from sklearn.model_selection import train_test_split

SEED = 42

dataset_train, dataset_test = train_test_split(
    dataset,
    test_size=0.2,
    random_state=SEED,
)

In [ ]:
dataset.shape, dataset_train.shape, dataset_test.shape

In [ ]:
import seaborn

seaborn.pairplot(dataset_train, hue='target')

## Pipelines

### Transformadores

<img src="transformadores.png" width=70%/>

In [ ]:
from sklearn.preprocessing import StandardScaler

# Exemplo de transformador
X_train = dataset_train.drop(columns='target')
y_train = dataset_train['target']

X_test = dataset_test.drop(columns='target')
y_test = dataset_test['target']

X_train.describe().round(2).transpose()

In [ ]:
import pandas as pd

# Cria o transformador.
scaler = StandardScaler()

# Treina o transformador.
scaler.fit(X_train)

# Aplica o transformador.
X_train_scaled = scaler.transform(X_train)

X_train_scaled = pd.DataFrame(
    X_train_scaled,
    columns=X_train.columns,
    index=X_train.index,
)

X_train_scaled.describe().round(2).transpose()

In [ ]:
# Fazendo coisa proibida só para ilustrar o StandardScaler.
# Isso é proibido AGORA, antes da modelagem. Mas DEPOIS da escolha de modelo,
# pode ser feito sem problemas.
X_test.describe().round(2).transpose()

In [ ]:
X_test_scaled = scaler.transform(X_test)

X_test_scaled = pd.DataFrame(
    X_test_scaled,
    columns=X_test.columns,
    index=X_test.index,
)

X_test_scaled.describe().round(2).transpose()

### Pipelines de transformação de dados

<img src="pipeline.png" width=70%/>

Para quem olha "de fora", uma *pipeline* se parece com um transformador e nada mais! Podemos treiná-lo com o método `fit`:

<img src="pipeline_transform_train.png" width=50%/>

E o que acontece "por trás das cortinas"? Deixa que o *Scikit-Learn* cuida de tudo para você! Mas, para nosso entendimento, eis o que acontece:

<img src="pipeline_transform_train_internals.png" width=50%/>

Agora que o pipeline está treinado, podemos usá-lo para fazer transformações de dados, como um transformer qualquer!

<img src="pipeline_transform_apply.png" width=50%>

E, por dentro, o que acontece? O *Scikit-Learn* vai chamar cada transformador, um por vez, em sequência:

<img src="pipeline_transform_apply_internals.png" width=50%/>

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# Cria a pipeline.
pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
])

pipe

In [ ]:
# Treina a pipeline.
pipe.fit(X_train)

In [ ]:
X_train_transformed = pipe.transform(X_train)

In [ ]:
X_train.shape, X_train_transformed.shape

In [ ]:
# Colunas na saida do PolynomialFeatures(degree=2, include_bias=False)

# Colunas de grau 1: as 13 features originais.
n_deg_1 = 13

# Colunas de grau 2 com features diferentes: 
# combinações de 2 features originais.
n_deg_2_diff = 13 * (13 - 1) // 2  # 13 escolha 2

# Colunas de grau 2 com features iguais:
# cada coluna original ao quadrado.
n_deg_2_same = 13

n_cols_total = n_deg_1 + n_deg_2_diff + n_deg_2_same

n_cols_total

### Pipelines preditoras

Toda *pipeline* é feita de uma sequência de estágios *transformadores*, e o último estágio pode ser um *transformador* ou um *modelo preditivo*.

- ***Pipeline*** **transformadora**:

Quando o último estágio é um *transformador* (como no exemplo anterior), a *pipeline* atua como um *transformador*

- ***Pipeline*** **preditora**:

Quando o último estágio é um *modelo preditivo*, a *pipeline* atua como um *modelo preditivo*

<img src="pipeline_predict.png" width=50%/> 

Treinamos a *pipeline* preditora do mesmo jeito que treinamos modelos:

<img src="pipeline_predict_train.png" width=50%>

"Por trás dos panos":

<img src="pipeline_predict_train_internals.png" width=80%>

Para fazer predições, use como se fosse um modelo qualquer!

<img src="pipeline_predict_apply.png" width=50%/>

Novamente, o que acontece "por dentro":

<img src="pipeline_predict_apply_internals.png" width=90%/>

### `ColumnTransformer`, `Pipeline` e transformadores

> ```Python
> geo_cols = [
>     'longitude',
>     'latitude',
> ]
> 
> numerical_cols = [
>     'housing_median_age',
>     'log_households',
>     'log_median_income',
>     'log_rooms_per_household',
>     'log_population_per_household',
>     'log_bedrooms_per_room',
> ]
> 
> categorical_cols = [
>     'ocean_proximity',
> ]
> 
> geo_pipeline = Pipeline([
>     ('imputer', SimpleImputer(strategy='median')),
>     ('cluster', KMeans(n_clusters=50)),
> ])
> 
> num_pipeline = Pipeline([
>     ('imputer', SimpleImputer(strategy='median')),
>     ('poly', PolynomialFeatures(degree=3, include_bias=False)),
>     ('scaler', StandardScaler()),
> ])
> 
> cat_pipeline = Pipeline([
>     ('encoder', OneHotEncoder(sparse_output=False)),
> ])
> 
> preprocessing_pipe = ColumnTransformer(
>     transformers=[
>         ('geo', geo_pipeline, geo_cols),
>         ('num', num_pipeline, numerical_cols),
>         ('cat', cat_pipeline, categorical_cols),
>     ],
>     remainder='passthrough',
> )
> ```

<img src="column_transformer.png" width=100%/>

## Comparação de modelos

### Versão simples: train-test-val split

### Versão mais sofisticada: validação cruzada

## Ajuste de hiperparâmetros

- `GridSearchCV`

## Regressão

- Medidas de desempenho: MSE, RMSE

- Análise de erros:

    - Resíduos

- Modelo trivial

## Classificação

- Classificação binária e multiclasse

    - Classificador trivial

De resto, só classificação binária.

- Medidas de desempenho:

    - acurácia

    - Precision e recall

        - trade-off precision vs. recall

        - Curva precision-recall, `cross_val_predict`

        - métrica F1

    - Sensibility e specificity

        - trade-off sensibility vs. specificity

        - Curva ROC, área sob a curva ROC (AUC ou AUROC)

## Modelo linear

- Definição

- Explicação intuitiva de porque não podemos ter *features colineares*